# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
pip install jupyter_contrib_nbextensions

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install jupyter_nbextensions_configurator

Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# store the csv in WeatherPy into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kangaatsiaq,68.3065,-53.4641,-14.53,98,100,11.61,GL,1644476513
1,1,rikitea,-23.1203,-134.9692,79.14,74,19,19.37,PF,1644476494
2,2,mahebourg,-20.4081,57.7000,80.58,69,10,7.29,MU,1644476518
3,3,barrow,71.2906,-156.7887,-16.58,63,100,6.91,US,1644476453
4,4,perth,-31.9333,115.8333,102.38,23,0,13.80,AU,1644476289


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [7]:
#heatmap of the humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()

#add the heatmap layer to the figure
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
idealWeatherDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & \
    (city_data_df["Wind Speed"] < 10) & \
    (city_data_df["Cloudiness"] == 0)
].dropna()

idealWeatherDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,kimberley,-28.7323,24.7623,78.49,47,0,1.14,ZA,1644476539
44,44,najran,17.4924,44.1277,70.38,22,0,3.56,SA,1644476627
82,82,port hueneme,34.1478,-119.1951,74.32,31,0,5.75,US,1644476686
86,86,myitkyina,25.3833,97.4000,79.95,28,0,2.80,MM,1644476734
130,130,sur,22.5667,59.5289,75.45,41,0,8.28,OM,1644476842


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = idealWeatherDF[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-9-baa509e1caa9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
21,kimberley,ZA,-28.7323,24.7623,
44,najran,SA,17.4924,44.1277,
82,port hueneme,US,34.1478,-119.1951,
86,myitkyina,MM,25.3833,97.4000,
130,sur,OM,22.5667,59.5289,
214,chandla,IN,25.0833,80.2000,
226,east london,ZA,-33.0153,27.9116,
244,moron,AR,-34.6534,-58.6198,
249,lashio,MM,22.9333,97.7500,
263,tecoanapa,MX,16.5167,-98.7500,


In [10]:
#set the parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
       
}

#Loop through the dataframe using interrows()
for index, row in hotel_df.iterrows():
    # get the Latitude and Longitude from the current row
    lat = row["Lat"]
    lng = row["Lng"]
    
    # use the latitude and longitude to set the location value in the params dictionary
    params["location"] = f"{lat},{lng}"
    
    #use the google apis places base url to search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print the url
    name_address = requests.get(base_url, params=params).json()
    
    #pprint(name_address) # print ["results"][0]["name"]
    
    try:
        # populate the hotel name
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print(f"Error finding lodging in {row['City']}. Skipping...")
        
hotel_df

Error finding lodging in kimberley. Skipping...
Error finding lodging in najran. Skipping...
Error finding lodging in port hueneme. Skipping...
Error finding lodging in myitkyina. Skipping...
Error finding lodging in sur. Skipping...
Error finding lodging in chandla. Skipping...
Error finding lodging in east london. Skipping...
Error finding lodging in moron. Skipping...
Error finding lodging in lashio. Skipping...
Error finding lodging in tecoanapa. Skipping...
Error finding lodging in sakassou. Skipping...
Error finding lodging in port elizabeth. Skipping...
Error finding lodging in bubaque. Skipping...
Error finding lodging in puerto escondido. Skipping...
Error finding lodging in turbat. Skipping...
Error finding lodging in lahan. Skipping...
Error finding lodging in ormara. Skipping...
Error finding lodging in karachi. Skipping...
Error finding lodging in bosaso. Skipping...
Error finding lodging in porto velho. Skipping...
Error finding lodging in ladybrand. Skipping...
Error fin

,City,Country,Lat,Lng,Hotel Name
21,kimberley,ZA,-28.7323,24.7623,
44,najran,SA,17.4924,44.1277,
82,port hueneme,US,34.1478,-119.1951,
86,myitkyina,MM,25.3833,97.4000,
130,sur,OM,22.5667,59.5289,
214,chandla,IN,25.0833,80.2000,
226,east london,ZA,-33.0153,27.9116,
244,moron,AR,-34.6534,-58.6198,
249,lashio,MM,22.9333,97.7500,
263,tecoanapa,MX,16.5167,-98.7500,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))